In [1]:
import time
from datetime import datetime
from tqdm.keras import TqdmCallback
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import LSTM, Dense, Dropout

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
import shutil
from sklearn.preprocessing import MinMaxScaler
from tqdm import tqdm_notebook as tqdm

import matplotlib.pyplot as plt

from darts import TimeSeries
from darts.dataprocessing.transformers import Scaler
from darts.models import RNNModel, TCNModel, TransformerModel, NBEATSModel, BlockRNNModel
from darts.utils.missing_values import fill_missing_values

from darts.metrics import mape, mse
from darts.utils.statistics import check_seasonality, plot_acf
from darts.datasets import AirPassengersDataset, SunspotsDataset
from darts.utils.timeseries_generation import datetime_attribute_timeseries

from math import sqrt

from datetime import timedelta

import warnings
warnings.filterwarnings("ignore")
import logging
logging.disable(logging.CRITICAL)

%pylab inline

Importing plotly failed. Interactive plots will not work.


Populating the interactive namespace from numpy and matplotlib


# The notebook leverages coinbase daily data
- open, low, high, close, volume
- unlike the other notebook where we have to create these daily candles

### Read in data

In [34]:
bitcoin_df = pd.read_csv("../data/historic_crypto_prices - bitcoin_jan_2017_sep_4_20201.csv", 
    index_col=0,
    parse_dates=True)

In [35]:
def convert_currency_to_float(input_df):
    for col in input_df.columns:
        input_df[col] = input_df[col].apply(lambda x: x.replace('$','').replace(",","")).astype(float)
    return input_df

In [36]:
bitcoin_df = convert_currency_to_float(bitcoin_df)

In [37]:
bitcoin_df

,open,high,low,close,volume,market_cap
date,,,,,,
2021-09-04,50009.33,50545.58,49548.78,49944.63,3.747133e+10,9.392861e+11
2021-09-03,49288.25,50982.27,48386.09,50025.37,4.320618e+10,9.407572e+11
2021-09-02,48807.85,50343.42,48652.32,49327.72,3.950807e+10,9.275909e+11
2021-09-01,47099.77,49111.09,46562.44,48847.03,3.913940e+10,9.185015e+11
2021-08-31,47024.34,48189.55,46750.09,47166.69,3.473036e+10,8.868699e+11
...,...,...,...,...,...,...
2017-01-05,1156.73,1191.10,910.42,1013.38,5.101990e+08,1.630025e+10
2017-01-04,1044.40,1159.42,1044.40,1154.73,3.449460e+08,1.857187e+10
2017-01-03,1021.60,1044.08,1021.60,1043.84,1.851680e+08,1.678637e+10


In [8]:
etherum_df = pd.read_csv("../data/historic_crypto_prices - etherum_jan_2017_sept_4_2021.csv", 
    index_col=0,
    parse_dates=True)

In [9]:
etherum_df.head()

,open,high,low,close,volume,market_cap
date,,,,,,
2021-09-04,"$3,937.91","$3,969.45","$3,837.93","$3,887.83","$20,806,963,328","$456,394,981,337"
2021-09-03,"$3,787.49","$4,022.47","$3,712.68","$3,940.61","$26,207,765,094","$462,538,485,147"
2021-09-02,"$3,825.03","$3,830.71","$3,726.75","$3,790.99","$24,387,397,330","$444,924,932,307"
2021-09-01,"$3,430.76","$3,836.87","$3,387.41","$3,834.83","$30,070,890,104","$450,017,458,312"
2021-08-31,"$3,227.76","$3,466.99","$3,195.22","$3,433.73","$27,280,502,987","$402,902,770,210"


In [39]:
etherum_df = convert_currency_to_float(etherum_df)

### Create the simulator

In [13]:
class BollingerBandsSimulator:

    def __init__(self, df_original, from_date, period, window, no_of_std, figsize=None):
        self.df = df_original
        self.from_date = from_date
        self.period = period
        self.window = window
        self.no_of_std = no_of_std
        self.figsize = figsize
        
        self.ml_model = None

        
        
    def _scale_time_series(self, input_df, train_cols=['open', 'high', 'low', 'Bollinger High','Rolling Mean'],
                           pred_col ='close', time_cols = ['year', 'month', 'day']):
        ts_transformers = {}
        ts_stacked_series = None
        train_series = None
        for col  in train_cols:
            transformer = Scaler()
            transformed_series = transformer.fit_transform(fill_missing_values(TimeSeries.from_series(input_df[col],  freq=self.period)))
            ts_transformers[col] = transformer    

            if ts_stacked_series:
                ts_stacked_series = ts_stacked_series.stack(transformed_series)

            else:
                ts_stacked_series = transformed_series
            
        
        # build year and month series:
        for col in time_cols:
            transformer = Scaler()
            transformed_series = transformer.fit_transform(datetime_attribute_timeseries(ts_stacked_series, attribute=col))
            ts_transformers[col] = transformer    

            ts_stacked_series = ts_stacked_series.stack(transformed_series)
             

        return ts_transformers, ts_stacked_series, TimeSeries.from_series(input_df[pred_col],  freq=self.period)
    
    def __check_ml_prediction(self, end_time, input_chunk_length=40, output_chunk_length=7, epochs=2) -> float:
        """train ML model to predict price movement over the next 7 days"""
        
        if not self.ml_model: # define model once, retrain incrementally
            self.ml_model = NBEATSModel(input_chunk_length=input_chunk_length, 
                                      output_chunk_length=output_chunk_length, n_epochs=epochs, random_state=0)

        training_df = self.df[self.df.index <= pd.to_datetime(end_time)]
        
        ts_transformers, ts_stacked_series, train_close_series = self._scale_time_series(training_df)
        

        # train the model
        self.ml_model.fit(series = train_close_series, past_covariates = [ts_stacked_series],  verbose=True)

        
        plt.show()
        pred_close = self.ml_model.predict(n=output_chunk_length, 
                                                 series=train_close_series, past_covariates = [ts_stacked_series])
        
        return pred_close.last_value()
        
    
#     def _build_candles_dataframe(self):
#         self.df = self.df.loc[self.from_date:, ["price"]].copy()
#         self.df = self.df["price"].resample(self.period).ohlc()
                
    def _build_bollinger_bands(self):
        rolling_mean = self.df['close'].rolling(self.window).mean()
        rolling_std = self.df['close'].rolling(self.window).std()

        self.df['Rolling Mean'] = rolling_mean
        self.df['Bollinger High'] = rolling_mean + (rolling_std * self.no_of_std)
        self.df['Bollinger Low'] = rolling_mean - (rolling_std * self.no_of_std)

    def _calculate_positions(self):
        self.df['Position'] = None
        self.df['Mode'] = None
        self.number_of_trades = 0
        buy_has_crossed_mean = False
        short_has_crossed_mean = False
        self.ml_prediction_date_and_price = {}


        mode = 'no_position'
        
        for index in range(len(self.df)):
            if index == 0:
                self.df.iloc[index, self.df.columns.get_loc('Mode')] =  mode
                continue

            row = self.df.iloc[index]
            prev_row = self.df.iloc[index - 1]
                                                           
            
            # get out of position if price crosses mean
            # (if mode == 'open' or mode == 'no_position' ) and 
            if mode == 'buy'  and row['close'] > row['Rolling Mean']:
                buy_has_crossed_mean = True
                
            if mode == 'short'  and row['close'] < row['Rolling Mean']:
                short_has_crossed_mean = True

            # open -> no_position? no position is below running mean
            # or, if we are above the top band (mean reversion)
            if mode == 'buy'  and ((row['close'] < row['Rolling Mean'] and buy_has_crossed_mean) or (row['close'] > row['Bollinger High'])):
                # check ML predicted trend as well
                ml_7_day_pred = self.__check_ml_prediction(row.name)
                if ml_7_day_pred < row['Rolling Mean']:
                    print("-------")
                    print('buy_to_no_position')
                    print(ml_7_day_pred, 'ml_7_day_pred')
                    print(row['Rolling Mean'], 'mean')
                

                    self.df.iloc[index, self.df.columns.get_loc('Position')] = 1

                    if index +1  == len(self.df):
                        self.df.iloc[index, self.df.columns.get_loc('Position')] = 0 
                    else:
                        self.df.iloc[index + 1, self.df.columns.get_loc('Position')] = 0 # for pct change it does a ffilll. ffill with zeros
                    self.df.iloc[index, self.df.columns.get_loc('Mode')] = 'buy_to_no_position'

                    mode = 'no_position'
                    buy_has_crossed_mean = False
            
            # close -> no_position? no position if above running mean
            # or, if we are below the bottom band (mean reversion)
            elif mode == 'short'  and ((row['close'] > row['Rolling Mean'] and short_has_crossed_mean) or (row['close'] < row['Bollinger Low'])):
                
                # check ML predicted trend as well
                ml_7_day_pred = self.__check_ml_prediction(row.name)
                if ml_7_day_pred > row['Rolling Mean']:
                    print("-------")
                    print('buy_to_no_position')
                    print(ml_7_day_pred, 'ml_7_day_pred')
                    print(row['Rolling Mean'], 'mean')
                    
                    self.df.iloc[index, self.df.columns.get_loc('Position')] = -1
                    if index +1  == len(self.df):
                        self.df.iloc[index, self.df.columns.get_loc('Position')] = 0 
                    else:
                        self.df.iloc[index + 1, self.df.columns.get_loc('Position')] = 0 # for pct change it does a ffilll. ffill with zeros
                    self.df.iloc[index, self.df.columns.get_loc('Mode')] = 'short_to_no_position'
                    mode = 'no_position'
                    short_has_crossed_mean = False

            # buy check with ML model
            elif mode == 'no_position' and row['close'] < row['Bollinger Low'] and prev_row['close'] > prev_row['Bollinger Low']:
                print('----------')
                print('buy')
                ml_7_day_pred = self.__check_ml_prediction(row.name)
                
                print(ml_7_day_pred, 'ml prediction 7 day')
                print(row.name, 'current date')
                print(row['Rolling Mean'], 'mean')
                
                if ml_7_day_pred > row['Rolling Mean']:
                    print('pred 7 day higher than mean taking position')
                    self.ml_prediction_date_and_price[row.name + timedelta(days=7)] = ml_7_day_pred #?? will this work
                
                    self.df.iloc[index, self.df.columns.get_loc('Position')] = 0 # buy. add one to index so that pct_change works

                    if index +1  == len(self.df):
                        self.df.iloc[index, self.df.columns.get_loc('Position')] = 1 
                    else:
                        self.df.iloc[index + 1, self.df.columns.get_loc('Position')] = 1 # buy. add one to index so that pct_change works
                    self.df.iloc[index, self.df.columns.get_loc('Mode')] = 'buy'
                    self.number_of_trades  +=1
                    mode = 'buy'     

            # short?
            elif mode == 'no_position' and  row['close'] > row['Bollinger High'] and prev_row['close'] < prev_row['Bollinger High']:
                print('----------')
                print('short')
                ml_7_day_pred = self.__check_ml_prediction(row.name)
                
                print(ml_7_day_pred, 'ml prediction 7 day')
                print(row.name, 'current date')
                print(row['Rolling Mean'], 'mean')
                
                if ml_7_day_pred < row['Rolling Mean']:
                    print('pred 7 day lower than mean taking position')
                    self.ml_prediction_date_and_price[row.name + timedelta(days=7)] = ml_7_day_pred 

                    self.df.iloc[index, self.df.columns.get_loc('Position')] = 0 #  short starts at the end of the day. Calculate pct_change starting tomorrow
                    self.df.iloc[index + 1, self.df.columns.get_loc('Position')] = -1 #  short starts at the end of the day. Calculate pct_change starting tomorrow
                    if index +1  == len(self.df):
                        self.df.iloc[index, self.df.columns.get_loc('Position')] = -1
                    else:
                        self.df.iloc[index + 1, self.df.columns.get_loc('Position')] = -1 # buy. add one to index so that pct_change works
                    self.df.iloc[index, self.df.columns.get_loc('Mode')] = 'short'
                    self.number_of_trades  +=1
                    mode = 'short' 
                       
            else:
                self.df.iloc[index, self.df.columns.get_loc('Mode')] = mode
        
    def _calculate_returns(self):
        
        self.df['Original Position'] = self.df['Position']
        self.df['Position'].fillna(method='ffill', inplace=True)
        self.df['Market Return'] = self.df['close'].pct_change()
        self.df['Strategy Return'] = self.df['Market Return'] * self.df['Position']

    def _plot_returns(self):
        self.df['Strategy Return'].cumsum().plot(figsize=self.figsize)
        

    def simulate(self):
#         self._build_candles_dataframe() we already have candles
        self._build_bollinger_bands()
        self._calculate_positions()
        self._calculate_returns()
        self._plot_returns()

        return (
            self.period, 
            self.window, 
            self.no_of_std, 
            self.df['Strategy Return'].sum(),
            self.number_of_trades
        )

In [14]:

simulator = BollingerBandsSimulator(
    bitcoin_df, 
    from_date="2017-1-01", 
    period="24H", 
    window=30, 
    no_of_std=1.5
)
simulator.simulate()

DataError: No numeric types to aggregate